# **Group Project 2: Data Structures in Action Table of Contents & Text Analysis**

Data Structures and Algorithms, Fall 2025

## **README**

**Group Number:**  

8

**Group Members:**  

Samanwita Mukherjee, Andrea Caceres, Chushmitha Battula, Sam Goodell

**Github Repo:**

https://github.com/lavender2412/GP2_DSA_Data_Structures_in_Action_Table_of_Contents_-_Text_Analysis.git

**Objectives:**

(1) Implement tree data structures and relevant methods to store, traverse, and organize a table of contents from a technical textbook.  
(2) Preprocess and analyze a public domain novel to investigate letter/word frequencies, bigrams/trigrams, and a variety of sentence structure metrics.

**Datasets:**

Question 1: Data Science and Predictive Analytics (2018) by Ivo D. Dinov (Table of Contents)

Question 2: Wuthering Heights (1847) by Emily Bronte

**Team Roles:**

Question 1 leads: Sam Goodell and Chushmitha Battula

Question 2 leads: Andrea Caceres and Samanwita Mukherjee


# **Question 1: Build a Table of Contents (Tree)**

**Task:** Create a hierarchical Table of Contents (TOC) from a Data Science / AI technical book

**Textbook Title:** Data Science and Predictive Analytics (2020)

**Author:** Ivo D. Dinov

## **Import and Clean Table of Contents:**

In [ ]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from collections import Counter
import string
import re as re

In [ ]:
!pip -q install gdown

import gdown, os

file_id = "17TUKK5KNXvE3tr3STZHNK_QL4Wpwcu_Z"
file_name = "table_of_contents_corrected.txt"

if not os.path.exists(file_name):
    gdown.download(id=file_id, output=file_name, quiet=False)

with open(file_name, encoding="utf-8") as file:
    contents = file.read()

cleaned_contents = []

#split text at each new line
for line in contents.splitlines():
    #remove leading and trailing spaces
    strip = line.strip()
    #remove all instances of 2 or more periods
    replace_period = re.sub(r'\.{2,}',"", strip)
    #create list by splitting at instances of 2 spaces
    split = replace_period.split("  ")
    #keep lists with lengths greater than 2, append
    if len(split) > 2:
        cleaned_contents.append(split)

Downloading...
From: https://drive.google.com/uc?id=17TUKK5KNXvE3tr3STZHNK_QL4Wpwcu_Z
To: /content/table_of_contents_corrected.txt
100%|██████████| 56.0k/56.0k [00:00<00:00, 24.0MB/s]


##**Create Node Class and Methods:**

In [ ]:
#create node class
class Node:
    def __init__(self, book, section, title, page):
        self.section = section
        self.title = title
        self.page = page
        self.book = book
        #create section order, count of periods in section + 1
        self.section_order = section.count(".") + 1 if section else 0
        self.total = 0
        self.children = []

    #assumes sorted table of contents
    def insert_section(self, section, title, page):
        new_section = Node(self.book, section, title, page)
        new_order = new_section.section_order
        #if section order is 1 greater than root/current node, appending to current node's children
        if new_order == self.section_order + 1:
            self.children.append(new_section)
            #increment total nodes
            self.total += 1
            return new_section
        #if not a direct child, loop through each child node
        for children in self.children:
            #check if child is in section lineage
            if section.startswith(children.section + "."):
                #if so, recurse with child node
                inserted = children.insert_section(section, title, page)
                #increment total nodes
                if inserted:
                    self.total+=1
                return inserted
        print(f"Could not add {new_section.title}, {new_section.section}")

    #loop through cleaned list of contents
    def insert_many(self, arr):
        for val in arr:
            self.insert_section(val[0], val[1], val[2])


    def print_toc(self, mode):
      # will store the values in list instead of printing them all
        lines = []
        def traverse(node, chapter_hierarchy, level):
          # wit
            if node.title:
              # if the node has an existing not none title, we concatenate the hierarchy to use it later in the numbered indentation
                number_str = ".".join(str(num) for num in chapter_hierarchy)
              # level helps with keeps track of how many indent positions should we apply based on the level in TOC
                indent = "  " * level
                if mode == "plain":
              # we just print the book TOC titles, when asked for plain formatting
                    lines.append(node.title)
                elif mode == "indented":
              # we use the pre-calculated indent variable and concatenate it to the title, when its asked to print with indentation format
                    lines.append(indent + node.title)
              # to print with number and indentation we use the above 2 pre-calculated variables to display the contents as requried
                elif mode == "numbered":
                    lines.append(indent + (f"{number_str} " if number_str else "") + node.title)

           # we loop over the children [], and perform preorder traversal here, to print the current node first and look into its children node, to maintain the TOC structure

            for i, child in enumerate(node.children, 1):

                traverse(child, chapter_hierarchy + [i], level + 1)

        # initial function call, that will execute the function

        traverse(self, [], 0)

        # returns the list of lines we required from the TOC, which will be used in the show_toc() method to display it correctly

        return lines

    def show_toc(book, mode):
      # method used to print the TOC in the colab cells without any interrupted
        toc_lines = book.print_toc(mode)
      # adds the line break after each line of the content to display it in html content
        html_block = "<br>".join(toc_lines)
        display(HTML(f"""<div style="max-height: 400px; overflow-y: auto; white-space: pre; font-family: monospace;">{html_block}</div>"""))


    def tree_height(self):
        #initialize empty array
        heights_arr = []
        #check for children of root/current node
        if self.children:
            #iterate over each child node
            for children in self.children:
                #recurse to each leaf
                height = children.tree_height()
                #after recursion, append height of each leaf
                heights_arr.append(height)
        #base case - return 0 because root node has height 0
        else:
            return 0
        # return max height + 1 to increment each recurion
        return 1 + max(heights_arr)


    def tree_depth(self, title):
        # using dfs approach here as we want to have the minimal traversal made to identify the actual title
        # we start with 0 at the root and the depth is 0 at root
        if self.book == title:
          return 0
        def dfs(node, depth):
          # check with title and return the depth when founded
            if node.title == title:
                return depth
            for child in node.children:
          # we loop through all the children [] and increase the depth by 1, with each recursive call
                result = dfs(child, depth + 1)
                if result is not None:
                    return result
            return None
        # this will call our dfs method
        return dfs(self, 0)


**Test Methods:**

In [ ]:
#Create root (book)
book = Node(book = "Data Science and Predictive Analytics", section = None, title = None, page = None)

#Insert all elements
book.insert_many(cleaned_contents)

#Show total elements in TOC
print(f"Total Sections in Table of Contents: {book.total}")

#Find tree height
print(f"Max Height in Table of Contents: {book.tree_height()}")

#Find depth with the title
print(f"Depth of Title: {book.tree_depth('Parkinson’s Disease')}")


Total Sections in Table of Contents: 579
Max Height in Table of Contents: 3
Depth of Title: 3


We can validate the above outputs, through the below show_toc() method.


In [ ]:
book.show_toc("plain")


In [ ]:
book.show_toc("indented")

In [ ]:
book.show_toc("numbered")

## **Discussion and Explanations**

**Tree Design**

We implemented a single node class with attributes describing the section (ex. "1.3.7"), title (ex. "Data Visualization"), page, total book sections, children, section_order, and book.  The section_order is deduced from the count of periods in each section string + 1, and describes the relative depth.

The book is identified as the root node, and its attribute "book" is inherited by each of the sections.

**Insertion Strategy**

To insert, the function first checks if it's section order is one more than that of the root (book) node.  If so, it immediately appends to the root's children attrribute and increments the tree's total sections.  If the inserted node is not a direct child, the function will loop through the children of the root node and check for a match in section lineage.  Once a match is identified, the insert function is recurively called and the total is updated.  An additional method, insert_many, calls insert_section for each item in an array of contents.

In this case, we are assuming that the table of contents are naturally sorted before insertion.  If a node were to be inserted before its parent node, the inserted section would not find a lineage match.  In such cases, the user would be prompted with a message indicating that the section could not be added to the contents.

In the best case, an individual insertion would have a time complexity of O(1) in the instance that the inserted node is a direct child of the root.  In the worst case (assuming a very unbalanced tree), the function will recurse through every node before finding its proper parent node, resulting in a time complexity of O(n).  However, in the average case, a relatively balanced tree will only require O(log n) comparisons before placing the inserted node.

**Traversal Method and Justification**

We have used the preoder traversal to print the Table of Contents in the use case. The requirement is to print the contents in order and since there is hierarchy and we want to make sure that chapter -> sub-chapter -> sub-sub-chapter order is maintained in the output, we want to make sure the root/ the parent node is printed first, before we traverse its children nodes.
Breadth-first search (BFS) would ideally first print all the chapters, the recursively consider one chapter at a time and print all its sub-chapters at all and all the sub-sub-chapters in the next iteration. This is not ideally considered to be the right approach to print the table of contents, since it will list all the available chapters first, all the sub-chapters and then go to all the sub-sub-chapters. Since we chose to consider pre-order traversal which is one of the techniques under Depth First Search (DFS) we are using a combination of pre-order + DFS methods here as our traversal methods for the Table of Contents (TOC).